In [8]:
import os
import pandapower as pp
import pandas as pd
import math

TIMESTEPS = 35040 # 365 days with 15min resolution
V_REF = 1.0 # pu
V_NOM = 416 # V

CURRENT_SCENARIO = '1_'

#load csv files
working_folder = os.getcwd()
root_dir = os.path.abspath(os.path.join(working_folder, os.pardir))
network_path = os.path.join(root_dir, 'data', CURRENT_SCENARIO, 'network')
data_path = os.path.join(root_dir, 'data', CURRENT_SCENARIO)

# network source
source_df = pd.read_csv(os.path.join(network_path,"source.csv"))
# substation transformer
transformer_df = pd.read_csv(os.path.join(network_path,"transformer.csv"))
# buses
buses_df = pd.read_csv(os.path.join(network_path,"buses.csv"))
# branches
lines_df = pd.read_csv(os.path.join(network_path,"lines.csv"))
#linecodes
linecodes = pd.read_csv(os.path.join(network_path,"linecodes.csv"))

# loads
loads_df = pd.read_csv(os.path.join(data_path,"load_data","load_info.csv"))
# EVs
evs_df = pd.read_csv(os.path.join(data_path,"ev_data","ev_info.csv"))
# generators
gens_df = pd.read_csv(os.path.join(data_path,"pv_data","pv_info.csv"))
# aggregate node consumption/generation data
aggregate_node_data = pd.read_csv(os.path.join(data_path,"aggregate_node_info.csv"))

In [11]:
#create empty net
net = pp.create_empty_network() 

#create buses
pp.create_bus(net, vn_kv=11.0, name="SourceBus", index=0, geodata=(buses_df.at[0, 'x'],buses_df.at[0, 'y']))
for index, row in buses_df.iloc[1:, :].iterrows():
    bus_name = row['name']       # f"Bus{index}"
    bus_geodata = (row['x'],row['y'])
    pp.create_bus(net, vn_kv=0.416, name=bus_name, geodata=bus_geodata)

#create external grid
grid_3ph_sc_power = math.sqrt(3) * 11 * 3 # MVA
pp.create_ext_grid(net, bus=0, vm_pu=V_REF, va_degree=0.0, s_sc_max_mva=grid_3ph_sc_power, rx_max=0.1, name="ExtGrid")

#create branch elements/transformers
pp.create_std_type(net, {"sn_mva": 0.8,
        "vn_hv_kv": 11,
        "vn_lv_kv": 0.416,
        "vk_percent": 4,
        "vkr_percent": 0.4,
        "pfe_kw": 1.18,
        "i0_percent": 0.1873,
        "shift_degree": 150,
        "vector_group": "Dyn5",
        "tap_side": "hv",
        "tap_neutral": 0,
        "tap_min": -2,
        "tap_max": 2,
        "tap_step_degree": 0,
        "tap_step_percent": 2.5,
        "tap_phase_shifter": False}, name='0.8 MVA 11/0.4 kV', element="trafo")
tid = pp.create_transformer(net, hv_bus=0, lv_bus=1, std_type="0.8 MVA 11/0.4 kV", name="Transformer")

#create branch elements/lines
alpha_al = 4.03e-3
alpha_cu = 3.93e-3
for _, row in linecodes.iterrows():
    line_name = str(row['name'])
    line_type = "ol" if row["type"] == "Overhead" else "cs"
    line_data = {"c_nf_per_km": 0, 
                 "r_ohm_per_km": row["r1"], 
                 "x_ohm_per_km": row["x1"], 
                 "max_i_ka": row["ampacity_a"] * 1e-3, 
                 "type": line_type, 
                 "alpha": alpha_al}
    pp.create_std_type(net, line_data, name=line_name, element='line')
for index, row in lines_df.iterrows():
    line_name = f"line{index+1}"
    line_fbus = row['from_bus']
    line_tbus = row['to_bus']
    # I multiply the length here to increase the line resistances!!!
    line_length = row['length'] * 4 * 1e-3 # convert from m to km
    line_std_type = row['linecode']
    pp.create_line(net, from_bus=line_fbus, to_bus=line_tbus, length_km=line_length, name=line_name, std_type=line_std_type)

#create loads
for index, row in aggregate_node_data.iterrows():
    load_name = row['load_name']
    load_bus = row['bus']
    load_p = row['load-ev_init_kW'] * 1e-3 # convert to mw
    load_q = 0
    pp.create_load(net, bus=load_bus, p_mw=load_p, q_mvar=load_q, name=load_name)

#create static generators
for index, row in gens_df.iterrows():
    gen_name = f"G{index+1}"
    gen_bus = row['bus']
    pp.create_sgen(net, bus=gen_bus, p_mw=0, q_mvar=0, name=gen_name)

In [12]:
# SAVE THE NETWORK TO JSON

pp.to_json(net, os.path.join(network_path,"IEEE_modified_LV_Feeder.json"))

In [6]:
# pp.diagnostic(net, report_style='compact', warnings_only=True)

In [ ]:
# pp.runpp(net)
# net.res_line.loading_percent.max()
# net.res_bus[net.bus.vn_kv==0.416].vm_pu.max()